In [3]:
import pandas as pd
import requests
import finnhub
import time
import urllib
from datetime import datetime
from datetime import date
from datetime import timedelta
pd.set_option('display.max_rows', None)
import pymongo

#user = db['user']
#symbols = db['symbols']
#app = db['app']
#fh = finnhub.Client(api_key="c5tec52ad3ifck7dehd0")
#dataLoc = '../Data/'
#favList = user.find_one({})['favorites']
#lastSyncDate = app.find_one({})['lastSyncDate']

In [8]:
username = "kxerxess"
password = "Theakatsuki99"
host = "cluster0.zvabl.mongodb.net"
db = "crypto_dashboard"

In [9]:
connection_string = (
            "mongodb+srv://"
            + username
            + ":"
            + urllib.parse.quote(password)
            + "@"
            + host
            + "/"
            + db
            + "?retryWrites=true&w=majority"
        )
client = pymongo.MongoClient(connection_string)
db = client['crypto_dashboard']

In [14]:
user = db['user']
app = db['app']
symbols = db['symbols']

In [22]:
symbols.delete_many({})

In [23]:
res = fh.crypto_symbols("BINANCE")
data = list()
daily = int(datetime(2020, 1, 1).timestamp())
hourly = int((datetime.now() - timedelta(days=20)).timestamp())
print(daily, hourly)

1577817000 1634075929


In [24]:
for each in res:
    display = each["displaySymbol"]
    symbol = each["symbol"]
    base = display.split("/")[0]
    quote = display.split("/")[1]
    name = base+quote
    if quote == "USDT":
        data.append(
            {
                "display": display,
                'name': name, 
                "base": base,
                "quote": quote,
                "symbol": symbol,
                "dailySync": daily,
                "hourlySync": hourly,
                "target": False
            }
        )
symbols.insert_many(data)

In [ ]:
for each in res:
    display = each["displaySymbol"]
    symbol = each["symbol"]
    base = display.split("/")[0]
    quote = display.split("/")[1]
    if quote == "USDT":
        data.append(
            {
                "display": display,
                "base": base,
                "quote": quote,
                "symbol": symbol,
                "dailySync": datetime.now()
            }
        )
symbols.insert_many(data)

In [ ]:
user.insert_one({
    'emailId': 'Kaustubhsharma999@gmail.com'
})

In [76]:
symbols.update_many({}, {'$set': {'dataSync': int(datetime(2020, 1, 1).timestamp())}})

In [84]:
def collect_day_data():
    for symbol in favList:

        currData = symbols.find_one({'base': symbol})
        try:
            df = pd.read_csv(dataLoc+symbol+'_1D.csv')
            exists = True
        except:
            df = None
            exists = False
        _from = currData['dataSync']
        till = int(datetime.now().timestamp())
        symbolSyntax = currData['symbol']
        res = fh.crypto_candles(symbolSyntax, 'D', _from, till)
        data = list()
        if res['s'] == 'no_data':
            print('Finished Sync for {0} with {1} rows'.format(symbol, len(data)))
            continue
        for count in range(len(res['t'])):
            rowDate = datetime.fromtimestamp(res['t'][count])
            data.append({
                'ts': res['t'][count],
                'date': rowDate,
                'dateDay': rowDate.day,
                'dateMonth': rowDate.month,
                'dateYear': rowDate.year,
                'open': res['o'][count],
                'close': res['c'][count],
                'high': res['h'][count],
                'low': res['l'][count],
                'volume': res['v'][count],
            })
        dfNew = pd.DataFrame(data)
        if exists:
            df = df.append(dfNew)
        else:
            df = dfNew.copy()
            
        df = df.drop_duplicates(subset=['ts'], keep='last')

        df.to_csv(dataLoc+symbol+'_1D.csv', index=False)

        symbols.update_one({'base': symbol}, {'$set': {'dataSync': max(res['t'])}})

        print('Finished Sync for {0} with {1} rows'.format(symbol, len(data)))

In [83]:
collect_day_data()

Starting Sync for BTC
Finished Sync for BTC with 1 rows
Starting Sync for ETH
Finished Sync for ETH with 1 rows
Starting Sync for ADA
Finished Sync for ADA with 1 rows
Starting Sync for SOL
Finished Sync for SOL with 1 rows
Starting Sync for XRP
Finished Sync for XRP with 1 rows
Starting Sync for MATIC
Finished Sync for MATIC with 1 rows
Starting Sync for BNB
Finished Sync for BNB with 1 rows
Starting Sync for LINK
Finished Sync for LINK with 1 rows
Starting Sync for LTC
Finished Sync for LTC with 1 rows
Starting Sync for EOS
Finished Sync for EOS with 1 rows
Starting Sync for BTT
Finished Sync for BTT with 1 rows
Starting Sync for TRX
Finished Sync for TRX with 1 rows
Starting Sync for VET
Finished Sync for VET with 1 rows
Starting Sync for QNT
Finished Sync for QNT with 1 rows
Starting Sync for CAKE
Finished Sync for CAKE with 1 rows
Starting Sync for HBAR
Finished Sync for HBAR with 1 rows


In [71]:
df = pd.read_csv('../Data/ADA'+'_1H.csv')

In [74]:
df.iloc[-1]

Unnamed: 0                    479
date          2021-10-31 15:30:00
dateDay                        31
dateMonth                      10
dateYear                     2021
dateHour                       15
dateMinute                     30
dateSecond                      0
open                        1.955
close                       1.944
high                        1.956
low                         1.939
volume                  2813367.9
Name: 479, dtype: object

In [69]:
def collect_hour_data():
    for symbol in favList:
    
        print('Starting Sync for {0}'.format(symbol))

        currData = symbols.find_one({'base': symbol})
        try:
            df = pd.read_csv(dataLoc+symbol+'_1H.csv')
            _from
            exists = True
        except:
            df = None
            exists = False
        today = datetime.now()
        till = int(today.timestamp())
        _from = int((datetime.now() - timedelta(days=20)).timestamp())
        symbolSyntax = currData['symbol']
        res = fh.crypto_candles(symbolSyntax, '60', _from, till)
        data = list()
        if res['s'] == 'no_data':
            print('Finished Sync for {0} with {1} rows'.format(symbol, len(data)))
            continue
        for count in range(len(res['t'])):
            date = datetime.fromtimestamp(res['t'][count])
            data.append({
                'date': date,
                'dateDay': date.day,
                'dateMonth': date.month,
                'dateYear': date.year,
                'dateHour': date.hour,
                'dateMinute': date.minute,
                'dateSecond': date.second,
                'open': res['o'][count],
                'close': res['c'][count],
                'high': res['h'][count],
                'low': res['l'][count],
                'volume': res['v'][count],
            })
        dfNew = pd.DataFrame(data)
        if exists:
            df = df.append(dfNew)
        else:
            df = dfNew.copy()

        df.to_csv(dataLoc+symbol+'_1H.csv')

        print('Finished Sync for {0} with {1} rows'.format(symbol, len(data)))

In [64]:
collect_hour_data()

Starting Sync for BTC
Finished Sync for BTC with 480 rows
Starting Sync for ETH
Finished Sync for ETH with 480 rows
Starting Sync for ADA
Finished Sync for ADA with 480 rows
Starting Sync for SOL
Finished Sync for SOL with 480 rows
Starting Sync for XRP
Finished Sync for XRP with 480 rows
Starting Sync for MATIC
Finished Sync for MATIC with 480 rows
Starting Sync for BNB
Finished Sync for BNB with 480 rows
Starting Sync for LINK
Finished Sync for LINK with 480 rows
Starting Sync for LTC
Finished Sync for LTC with 480 rows
Starting Sync for EOS
Finished Sync for EOS with 480 rows
Starting Sync for BTT
Finished Sync for BTT with 480 rows
Starting Sync for TRX
Finished Sync for TRX with 480 rows
Starting Sync for VET
Finished Sync for VET with 480 rows
Starting Sync for QNT
Finished Sync for QNT with 480 rows
Starting Sync for CAKE
Finished Sync for CAKE with 480 rows
Starting Sync for HBAR
Finished Sync for HBAR with 480 rows


In [7]:
fh = finnhub.Client(api_key="c5tec52ad3ifck7dehd0")
for each in fh.crypto_symbols('BINANCE'):
    display = each['displaySymbol']
    symbol = each['symbol']
    base = display.split('/')[0]
    quote = display.split('/')[1]
    if quote == 'USDT':
        print(base, quote, symbol)

ARDR USDT BINANCE:ARDRUSDT
DNT USDT BINANCE:DNTUSDT
KEY USDT BINANCE:KEYUSDT
USDP USDT BINANCE:USDPUSDT
FRONT USDT BINANCE:FRONTUSDT
GALA USDT BINANCE:GALAUSDT
ETH USDT BINANCE:ETHUSDT
TVK USDT BINANCE:TVKUSDT
AGLD USDT BINANCE:AGLDUSDT
AR USDT BINANCE:ARUSDT
RAY USDT BINANCE:RAYUSDT
STRAX USDT BINANCE:STRAXUSDT
BKRW USDT BINANCE:BKRWUSDT
NKN USDT BINANCE:NKNUSDT
EOSUP USDT BINANCE:EOSUPUSDT
BTG USDT BINANCE:BTGUSDT
TRIBE USDT BINANCE:TRIBEUSDT
LTCUP USDT BINANCE:LTCUPUSDT
ONT USDT BINANCE:ONTUSDT
DGB USDT BINANCE:DGBUSDT
BAT USDT BINANCE:BATUSDT
TUSD USDT BINANCE:TUSDUSDT
XTZUP USDT BINANCE:XTZUPUSDT
IRIS USDT BINANCE:IRISUSDT
MINA USDT BINANCE:MINAUSDT
ONG USDT BINANCE:ONGUSDT
LINKUP USDT BINANCE:LINKUPUSDT
ASR USDT BINANCE:ASRUSDT
UMA USDT BINANCE:UMAUSDT
QNT USDT BINANCE:QNTUSDT
TRX USDT BINANCE:TRXUSDT
ALPHA USDT BINANCE:ALPHAUSDT
TKO USDT BINANCE:TKOUSDT
FET USDT BINANCE:FETUSDT
ELF USDT BINANCE:ELFUSDT
TROY USDT BINANCE:TROYUSDT
SFP USDT BINANCE:SFPUSDT
BNT USDT BINANCE:BNTUSDT
